In [1]:
import numpy as np
import os
import sys
import torch
from torch.profiler import profile, record_function, ProfilerActivity

sys.path.append(os.path.join(os.getenv("HOME"), "RNN_Manifold/"))
from manifold_encoder_decoder import s1_direct_product_decoder, encoder_decoder_core


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using {}".format(device))

using cuda


The purpose of this notebook is to get an idea of the resource use of the main decoder functions to make sure nothing weird is going on.

In [3]:
batch_size = 1000
embedded_dim = 20
manifold_dim = 1
decoder_hidden_size = 1000
encoder_hidden_size = int(1.5 * decoder_hidden_size)
decoder_n_hidden = 2
encoder_n_hidden = decoder_n_hidden

fake_data = np.random.uniform(-1, 1, (batch_size, embedded_dim))
fake_data_torch = torch.tensor(fake_data, dtype=torch.get_default_dtype()).to(device)

In [4]:
encoder_net = encoder_decoder_core.AllPeriodicEncoder(embedded_dim, manifold_dim, encoder_hidden_size, encoder_n_hidden).to(device)
decoder_net = encoder_decoder_core.AllPeriodicDecoder(embedded_dim, manifold_dim, decoder_hidden_size, decoder_n_hidden).to(device)

/home/trevor/brainvenv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [8]:
with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
    decoded_points, decoded_angles, re_encoded_points, _ = s1_direct_product_decoder.decode_encode_cost(decoder_net, encoder_net, fake_data_torch)

print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=20))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             aten::tanh         4.25%     122.000us         6.34%     182.000us      22.750us      69.000us         7.24%      69.000us       8.625us           0 b           0 b      30.01 Mb      30.01 M

STAGE:2023-01-17 16:43:24 76944:76944 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-17 16:43:24 76944:76944 ActivityProfilerController.cpp:300] Completed Stage: Collection


In [10]:
with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
    _ = s1_direct_product_decoder.distance_costs(encoder_net, re_encoded_points, decoded_angles, 20)

print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=20))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             aten::tanh         0.13%      59.000us         1.21%     564.000us     141.000us       1.972ms         6.87%       1.972ms     493.000us           0 b           0 b     686.65 Mb     686.65 M

STAGE:2023-01-17 16:44:02 76944:76944 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-17 16:44:02 76944:76944 ActivityProfilerController.cpp:300] Completed Stage: Collection
